In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

In [4]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

In [5]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [7]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [8]:
from pykinect2 import PyKinectV2
from pykinect2.PyKinectV2 import *
from pykinect2 import PyKinectRuntime
import numpy as np
import cv2
import ctypes
import _ctypes
import sys

kinect = PyKinectRuntime.PyKinectRuntime(PyKinectV2.FrameSourceTypes_Color | PyKinectV2.FrameSourceTypes_Depth | PyKinectV2.FrameSourceTypes_Body)

def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(x, y)
    if event == cv2.EVENT_RBUTTONDOWN:
        Pixel_Depth = frameD[((y * 512) + x)]
        print(Pixel_Depth)
                
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    while True:
        
        # --- Getting frames and drawing
        if kinect.has_new_color_frame():
            image_np = kinect.get_last_color_frame()
            image_np = np.reshape(image_np, (kinect.color_frame_desc.Height, kinect.color_frame_desc.Width, 4))
            frameC = kinect._color_frame_data
            image_np = cv2.cvtColor(image_np, cv2.COLOR_RGBA2RGB)
            
            # Definite input and output Tensors for detection_graph
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
       
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=8)
            
            # Get coordinates of detected boxes ; ymin, ymax, xmin, xmax
            coordinates = vis_util.return_coordinates(
                        image_np,
                        np.squeeze(boxes),
                        np.squeeze(classes).astype(np.int32),
                        np.squeeze(scores),
                        category_index,
                        use_normalized_coordinates=True,
                        line_thickness=8,
                        min_score_thresh=0.80)

      
            '''filename_string = 'class_coordinate'
            textfile = open("file/"+filename_string+".txt", "a")
            textfile.writelines(pickle.dump(coordinates))
            textfile.write("\n")'''
            '''with open('file/class_coordinate.txt', 'wb') as f:
            pickle.dump(coordinates, f)'''
            print(*coordinates)

  
            '''box_to_color_map = collections.defaultdict(list)
            coordinates_list = []
            counter_for = 0
            for box, color in box_to_color_map.items():
                ymin, xmin, ymax, xmax = box
                height, width, channels = image.shape
                ymin = int(ymin*height)
                ymax = int(ymax*height)
                xmin = int(xmin*width)
                xmax = int(xmax*width)
                coordinates_list.append([ymin, ymax, xmin, xmax])
                counter_for = counter_for + 1
            print(coordinates_list)'''
            
            # The following code replaces the 'print ([category_index...' statement
            '''objects = []
            for index, value in enumerate(classes[0]):
                object_dict = {}
                if scores[0, index] > 0.9:
                    object_dict[(category_index.get(value)).get('name').encode('utf8')] = scores[0, index]
                    objects.append(object_dict)
                    print(objects)'''
            
            # VideoFileOutput.write(image_np)
            cv2.imshow('KINECT Color', image_np)
            cv2.waitKey(1)
        
        if kinect.has_new_depth_frame():
            frame = kinect.get_last_depth_frame()
            frameD = kinect._depth_frame_data
            frame = frame.astype(np.uint8)
            frame = np.reshape(frame, (424, 512))
            frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)

            ##output = cv2.bilateralFilter(output, 1, 150, 75)
            cv2.imshow('KINECT Video Stream', frame)
            cv2.setMouseCallback('KINECT Video Stream', click_event)
            output = None

        key = cv2.waitKey(1)
        if key == 27: break















person 36 906 593 1116 95.11246681213379
person 28 884 674 1130 82.23991394042969





person 11 1003 1000 1915 81.92248344421387
person 132 873 761 1117 88.18260431289673
person 13 986 975 1913 81.19440078735352




person 21 991 879 1909 82.07733631134033

person 21 1011 938 1914 85.58282852172852



person 21 983 959 1915 84.01384949684143
person 22 981 941 1915 84.05178785324097
person 17 983 948 1914 81.93680047988892



person 20 982 963 1913 82.52840042114258
person 17 982 942 1914 84.41942930221558



person 26 1018 946 1913 81.96154832839966



person 138 885 842 1183 80.15549778938293
person 173 872 849 1200 92.86049008369446
person 174 870 849 1204 96.14723920822144
person 178 868 840 1210 97.50559329986572
person 75 1028 1146 1920 82.78453350067139
person 171 865 841 1201 98.05779457092285
person 168 866 842 1201 98.06301593780518
person 171 867 842 1200 98.29610586166382
person 77 992 1152 1920 81.83531761169434
person 171 869 841 1208 98.31665754318237
person

person 0 918 225 1091 91.769939661026
person 0 927 205 1087 92.67818927764893
person 0 931 205 1099 91.54974222183228
person 0 927 213 1091 91.75944328308105
person 0 921 217 1083 92.88082122802734
person 0 924 204 1095 91.89159870147705
person 0 922 214 1083 92.30842590332031
person 0 921 208 1098 92.78784990310669
person 0 922 205 1095 92.54049062728882
person 1 919 210 1092 92.32029914855957
person 0 918 206 1094 92.85520315170288
person 0 917 212 1093 91.28729701042175
person 0 919 211 1096 92.33474731445312
167 122
person 0 918 211 1092 91.990065574646
person 0 919 211 1093 91.19471311569214
person 0 918 214 1094 91.69235229492188
person 0 918 216 1092 91.88384413719177
person 0 921 218 1089 91.07693433761597
person 0 928 216 1092 91.19551181793213
person 0 927 210 1092 91.78626537322998
person 0 926 209 1093 91.6517972946167
person 0 925 211 1093 90.9074068069458
person 0 928 209 1095 89.41864967346191
person 0 932 214 1081 89.05045986175537
person 0 925 224 1089 88.7169182300567

KeyboardInterrupt: 